In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Feature Engineering

In [3]:
df = pd.read_csv("cleaned_training_set_VU_DM.csv")
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,bool_visitor_hist
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
3,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,27348,2,4.0,...,0.0,0.0,0.0,-1.0,0.0,5.0,0,0.0,0,0
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0


In [4]:
df_test = pd.read_csv("cleaned_test_set_VU_DM.csv")

## Find columns with numeric values so that we can caluculate mean, std, median per prop_id


In [5]:
combined_df = pd.concat([df, df_test], ignore_index=False)

In [6]:
bool_flag_cols = [col for col in df.columns if 'bool' in col.lower() or 'flag' in col.lower()]

# transform the columns to boolean
for col in bool_flag_cols:
    df[col] = df[col].astype(bool)


In [7]:
print(df['date_time'].dtypes)
# transform it into datetime
df['date_time'] = pd.to_datetime(df['date_time'])


object


In [8]:
# print columns end with rate and env
rate_cols = df.columns[df.columns.str.endswith('rate')]
inv_cols = [col for col in df.columns if 'inv' in col.lower()]
print(rate_cols)
print(inv_cols)

Index(['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate',
       'comp6_rate', 'comp7_rate', 'comp8_rate'],
      dtype='object')
['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']


In [9]:
numeric_cols = [col for col in df.select_dtypes(include=['int64', 'float64']).columns if 'id' not in col and col not in rate_cols and col not in inv_cols]
print(numeric_cols)
print(len(numeric_cols))

['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff', 'gross_bookings_usd']
25


In [10]:
features = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff']

grouped = combined_df.groupby(['prop_id'])
stats = grouped[features].agg(['mean', 'median'])

original_train_len = len(df)
original_test_len = len(df_test)
for feature in features:
    for id_col in ['prop_id']:
        for stat in ['mean', 'median']:
            column_name = f'{feature}_{id_col}_{stat}'
            combined_df[column_name] = grouped[feature].transform(stat)

In [11]:
# People per room
combined_df['people_per_room'] = (combined_df['srch_adults_count'] + combined_df['srch_children_count']) / combined_df['srch_room_count']

# Interaction: people per room with location score
combined_df['people_location_interaction'] = combined_df['people_per_room'] * combined_df['prop_location_score1']


In [12]:
# Interaction: historical average daily rate with current price
combined_df['hist_price_interaction'] = combined_df['visitor_hist_adr_usd'] / combined_df['price_usd']

In [13]:
combined_df['rating_review_interaction'] = combined_df['prop_starrating'] * combined_df['prop_review_score']
combined_df['price_location_ratio'] = combined_df['price_usd'] / combined_df['prop_location_score1']

In [14]:
df = combined_df.iloc[:original_train_len]
df_test = combined_df.iloc[original_train_len:original_train_len + original_test_len]

In [16]:
df_test.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [17]:
# List of features to remove
features_to_remove = [
    'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
    'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
    'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
    'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
    'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
    'comp8_rate_percent_diff'
]

# Assuming df is your DataFrame
df = df.drop(columns=features_to_remove)

# Now df will no longer contain the columns listed in features_to_remove


In [18]:
df_test = df_test.drop(columns=features_to_remove)


In [19]:
df.to_csv('feature_engineered_training_set_VU_DM.csv', index=False)

In [20]:
df_test.to_csv('feature_engineered_test_set_VU_DM.csv', index=False)

In [21]:
# truncate into chunks
# Load the large CSV file
file_path = 'feature_engineered_training_set_VU_DM.csv'
data = pd.read_csv(file_path)

# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(data), chunk_size):
    chunk = data.iloc[i:i + chunk_size]
    chunk.to_csv(f'feature_engineered_training_chunk_{i//chunk_size}.csv', index=False)